# Precificação de Ativos Financeiros com Risco Condicional

## Definição do Problema

## Instalando Pacotes Para Manipulação de Dados

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
# !pip install -q -U watermark

In [ ]:
# Imports
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Henrique Krupck" --iversions

## Construindo a Arquitetura do Modelo

http://www.deeplearningbook.com.br/ (Capítulos de 1 a 39)

![title](imagens/imagem1.png)

## Dense Layer (Camada Densa)

A saída de cada camada é a transformação da saída das camadas anteriores. 

Estamos tentando maximizar a probabilidade de forma equivalente tentando minimizar a probabilidade logarítmica negativa. Assim, atualizamos os parâmetros da rede com derivadas parciais da função de perda com relação a cada parâmetro. Escalamos por uma taxa de aprendizagem para evitar que o gradiente salte e não se acomode nos mínimos locais mais baixos possíveis.

Para obter os gradientes da função de perda, usamos a regra da cadeia (chain rule):

$$\frac{\partial L}{\partial x_i} = \sum_j \frac{\partial L}{\partial y_j}\frac{\partial y_j}{\partial x_i}$$

Uma vez que cada entrada de uma determinada camada é saída da anterior, podemos armazenar a derivada de entrada de cada camada e transferi-la para a camada anterior durante o backpropagation.

Usando a regra da cadeia, obtemos as fórmulas necessárias que precisamos para atualizar os parâmetros da rede.

Considerando a camada de saída, os gradientes de que precisamos:

***Derivada em relação à entrada (input):***
$$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \omega^T$$

***Derivada em relação à matriz de peso:***
$$\frac{\partial L}{\partial \omega} = X^T \frac{\partial L}{\partial Y}$$

***Derivada em relação ao vetor de polarização (bias):***
$$\frac{\partial L}{\partial B} = \frac{\partial L}{\partial Y}$$

A classe abaixo implementa uma camada totalmente conectada (Densa):

- O método Forward refere-se à operação linear que produz saída que será transferida para a função de ativação que corresponderá à próxima camada.
 
 
- O método backward refere-se a backpropagation, que toma a derivada da função de perda em relação à saída da camada como argumento, calcula as derivadas em relação à sua própria entrada e pesos, atualiza seus pesos e vieses e, em seguida, retorna a derivada da função de perda em relação à sua própria entrada

In [ ]:
# Classe para a camada densa
class Dense:
    
    # Método construtor
    def __init__(self, feat_size, out_size):
        self.feat_size = feat_size
        self.out_size = out_size
        self.weights = (np.random.normal(0, 1, feat_size * out_size) * np.sqrt(2 / feat_size)).reshape(feat_size, out_size)
        self.bias = np.random.rand(1, out_size) - 0.5

    # Método da passada linear para frente
    def forward(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return(self.output)

    # Método da passada de volta (backpropagation)
    def backward(self, output_der, lr): 
        input_der = np.dot(output_der, self.weights.T)
        weight_der = np.dot(self.input.T.reshape(-1, 1), output_der)
        self.weights -= lr * weight_der
        self.bias -= lr * output_der
        return(input_der)

## Função de Ativação

Em vez de implementar a função de ativação dentro da camada densa, a implementação como camada separada simplifica o backpropagation. Esta camada não atualizará nenhum parâmetro, apenas retornará a derivada da função de perda em relação à função de ativação para a camada anterior totalmente conectada.

Na passagem para a frente, a camada de ativação pegará a saída da camada densa e a transferirá após a aplicação da função ReLu.

In [ ]:
# Função de ativação
def relu(x):  
    return(np.maximum(0, x))

In [ ]:
# Derivada da função de ativação
def relu_prime(x):  
    x[x > 0] = 1
    x[x <= 0] = 0  
    return x

In [ ]:
# Classe da camada de ativação
class ActLayer:
    
    # Método construtor
    def __init__(self, act, act_prime):
        self.act = act
        self.act_prime = act_prime

    # Recebe a entrada (input) e retorna a saída da função de ativação
    def forward(self, input_data):
        self.input = input_data
        self.output = self.act(self.input)
        return(self.output)

    # Observe que não estamos atualizando nenhum parâmetro aqui
    # Usamos a taxa de aprendizagem como parâmetro porque definiremos o método de ajuste de uma forma 
    # que todas as camadas o exigirão.
    def backward(self, output_der, lr):
        return(self.act_prime(self.input) * output_der)

## Função de Perda e Derivada

Diversas funções de perda podem ser usadas dependendo se o modelo é de classificação ou regressão. Aqui usaremos a função mais comum em regressão, a MSE.

In [ ]:
# Usaremos a Mean-Squared-Error como função de perda
def mse(y_true, y_pred):
    return(np.mean((y_pred - y_true)**2))

In [ ]:
# Derivada da função de perda
def mse_prime(y_true, y_pred):
    return(2*(y_pred - y_true) / y_true.size)

## Classe Final do Modelo

Observe que fazemos o loop nas camadas em ordem reversa.

In [ ]:
# Modelo
class Network:
    
    # Método construtor
    # Inicializa com a função de perda e sua derivada
    def __init__(self, loss, loss_prime):  
        self.layers = []  
        self.loss = loss
        self.loss_prime = loss_prime

    # Método para adicionar camadas ao grafo computacional
    def add(self, layer):
        self.layers.append(layer)

    # Implementando apenas forward-pass para predição
    def predict(self, input_data):
        
        # Lista para o resultado
        result = [] 

        for a in range(len(input_data)):
            
            # Camada de saída
            layer_output = input_data[a]
            
            # Loop pelas camadas
            for layer in self.layers:
                
                # Movendo vetores de camada para camada
                layer_output = layer.forward(layer_output)
                
            result.append(layer_output)

        return(result)

    # Método de treinamento
    def fit(self, X_train, y_train, epochs, lr):

        # Número de iterações
        for a in range(epochs):  
            
            # Inicializa a variável de cálculo do erro
            err = 0

            # Temos 1 passagem para a frente e para trás para cada ponto de dados 
            # Esse algoritmo de aprendizagem usa a Descida Estocástica do Gradiente
            for j in range(len(X_train)):
                
                # Camada de saída
                layer_output = X_train[j]
                
                # Loop pelas camadas
                for layer in self.layers:
                    layer_output = layer.forward(layer_output)

                # Vamos guardar o erro e mostrar durante o treinamento
                err += self.loss(y_train[j], layer_output)

                # Observe que fazemos o loop nas camadas em ordem reversa.
                # Inicialmente calculamos a derivada da perda com relação à previsão.
                # Em seguida, a camada de saída irá calcular a derivada em relação à sua entrada
                # e irá passar esta derivada de entrada para a camada anterior que corresponde à sua derivada de saída
                # e essa camada repetirá o mesmo processo, passando sua derivada de entrada para a camada anterior.

                # dL/dY_hat
                gradient = self.loss_prime(y_train[j], layer_output)  
                
                # Este loop é a razão de termos dado lr à camada de ativação como argumento
                for layer in reversed(self.layers):
                    
                    # Definindo gradiente para dY / dh_ {i + 1} da camada atual
                    gradient = layer.backward(gradient, lr)

            err /= len(X_train)
            
            print('Epoch %d/%d   Erro = %f' % (a + 1, epochs, err))

## Vamos Testar o Modelo Resolvendo o XOR (Ou Exclusivo da Lógica Computacional)

In [ ]:
# Dados
x_train = np.array([[[0, 0]], [[0, 1]], [[1, 0]], [[1, 1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# Ajuste dos dados
x_train = x_train.reshape(-1, 2)
y_train = y_train.reshape(-1, 1)

# Modelo
modelo_xor = Network(mse, mse_prime)
modelo_xor.add(Dense(2, 3))
modelo_xor.add(ActLayer(relu, relu_prime))
modelo_xor.add(Dense(3, 1))

# Treinamento:
modelo_xor.fit(x_train, y_train, epochs = 2000, lr = 0.01)

# Teste
y_pred = modelo_xor.predict(x_train)

In [ ]:
print("Valor Real:", "\n",
      list(y_train.reshape(-1,)), "\n",
      "------------", "\n",
      "Valor Previsto:", "\n",
      [round(float(a)) for a in y_pred])

## Precificação de Ativos Financeiros com Risco Condicional

## Gerando Preços de Ações

O modelo Black Scholes, também conhecido como modelo Black-Scholes-Merton (BSM), é um modelo matemático para precificar um contrato de opções (ações de empresas). Em particular, o modelo estima a variação ao longo do tempo de instrumentos financeiros. Ele pressupõe esses instrumentos (como ações ou futuros) terão uma distribuição lognormal de preços. Usando essa suposição e levando em consideração outras variáveis importantes, a equação deriva o preço de uma opção de compra.

O modelo assume que o preço dos ativos fortemente negociados segue um movimento browniano geométrico com flutuação e volatilidade constantes. Quando aplicado a uma opção de ações, o modelo incorpora a variação constante do preço da ação, o valor do dinheiro no tempo, o preço de exercício da opção e o tempo para o vencimento da opção, criando assim um risco condicional para o investimento.

Também chamado de Black-Scholes-Merton, foi o primeiro modelo amplamente utilizado para precificação de opções. É utilizado para calcular o valor teórico das opções usando os preços atuais das ações, dividendos esperados, preço de exercício da opção, taxas de juros esperadas, prazo de vencimento e expectativa volatilidade.

A fórmula, desenvolvida por três economistas - Fischer Black, Myron Scholes e Robert Merton - é talvez o modelo de precificação de opções mais conhecido do mundo. A equação inicial foi introduzida no artigo de Black and Scholes de 1973, "The Pricing of Options and Corporate Liability", publicado no Journal of Political Economy. Black faleceu dois anos antes de Scholes e Merton receberem o Prêmio Nobel de Economia de 1997 por seu trabalho na descoberta de um novo método para determinar o valor dos derivados (o Prêmio Nobel não é concedido postumamente; no entanto, o comitê do Nobel reconheceu o papel de Black no modelo Black-Scholes).

Referências:

https://www.investopedia.com/terms/b/blackscholes.asp

***Equação diferencial estocástica de Black e Scholes para preços de ações:***

$$dS_t = S_0(\sigma dB_t + rdt)$$

***A solução é dada por:***

$$S_t = S_0e^{\sigma(B_t - B_0) + (r - \frac{1}{2}\sigma^2)t}$$

***Discretizando:***

$$log(S_t) - log(S_0) = \sigma N(0, t) + (r - \frac{1}{2}\sigma^2)t$$

***$B_t$*** : Brownian Motion

***$S_i$*** : Stock price at time i

***$r$*** : Risk Free rate

***$\sigma$*** : Predicted Volatility, taken as constant for simplecity

***Equação diferencial parcial de Black e Scholes para preços de opções:***

$$rF(t, S_t) = \frac{\partial F}{\partial t}(t, S_t) + rS_t\frac{\partial F}{\partial x}(t, S_t) + \frac{1}{2}\sigma^2 S_t^2\frac{F}{x^2}(t, S_t)$$

***A solução é dada por:***

$$C_0 = S_0\mathcal{N}(d_1) - e^{-rT}K\mathcal{N}(d_2)$$

A solução pode ser obtida de 2 maneiras: 

- Primeiro usando a abordagem da árvore bionomial, indo do caso discreto para o contínuo. 

- Segundo resolvendo o PDE por várias mudanças de variáveis, então obtemos a equação.

Mais detalhes, consulte:

Primeiro caso: Cox-Rubinstein, OPTION PRICING: A SIMPLIFIED APPROACH

Segundo caso: Black-Scholes The Pricing of options and Corporate Liabilities.

## Prepara as variáveis

In [ ]:
# Volatility (standard deviation), sigma de SDE e PDE
vol = 0.17 

In [ ]:
# Maturity
T = 1/2 

In [ ]:
#  Número de etapas que usamos para discretizar o processo acima
n = 1000

In [ ]:
# O preço inicial do estoque corresponde a S_0 nas funções acima
s_0 = 100  

In [ ]:
#  Taxa livre de risco, termo que deriva de SDE -> r
r = 0.05  

In [ ]:
# Preço de exercício da opção -> K
k = 100 

## Funções de Cálculo Para Geração de Dados

In [ ]:
def calculate_spot(prev, sigma, r, step, random):
    return(prev + (sigma*prev*random) + (r*prev*step))

In [ ]:
def sim_spot(s0, r, steps, maturity, vol):
    delta_t = T/steps
    time = np.round(np.arange(0, maturity+delta_t, delta_t), 4)  
    prices = [s0]
    normal_dist = np.random.normal(0, np.sqrt(delta_t), 10000)
    for a in range(steps):
        prices.append(calculate_spot(prices[-1], vol, r, delta_t, normal_dist[a]))
    return(prices)

In [ ]:
# Gerando 5 caminhos diferentes para testar as funções
# Vamos usar apenas 1 caminho no treinamento de rede
sims = pd.DataFrame()
for a in range(5):
    sims[a] = sim_spot(s_0, r, n, T, vol)

In [ ]:
# Valores para simulações
sims.columns = ["Sim_1", "Sim_2", "Sim_3", "Sim_4", "Sim_5"]
sims.index = np.round(np.arange(0, 0.5 + (0.5 / 1000), 0.5 / 1000), 4)

In [ ]:
sns.set(style = "whitegrid", font_scale = 2.5)
plt.figure(figsize = (40, 18))
ax = sns.lineplot(data = sims, palette = "bright", linewidth = 2.7)
ax.set(xlabel = 'Passos', ylabel = 'Preços dos Ativos', title = "Simulações")

## Preparando os Preços Finais

In [ ]:
def d1(s, k, r, t, T, vol): 
    if T != t:
        nomin = np.log(s/k) + (r + 0.5*(vol**2))*(T-t)
        denom = vol*np.sqrt((T-t))
        return(nomin/denom)
    else:
        None


def d2(s, k, r, t, T, vol): 
    if T != t:
        nomin = np.log(s/k) + (r - 0.5*(vol**2))*(T-t)
        denom = vol*np.sqrt((T-t))
        return(nomin/denom)
    else:
        None


def call(d1, d2, k, r, T, t, s):
    return(s*scipy.stats.norm.cdf(d1) - k*np.exp(-r*(T-t))*scipy.stats.norm.cdf(d2))

In [ ]:
call_prices = []
maturity = []
for (a, b) in zip(sims["Sim_1"], sims.index):
    if b != T:
        d1_ = d1(a, k, r, b, T, vol)
        d2_ = d2(a, k, r, b, T, vol)
        call_prices.append(call(d1_, d2_, k, r, T, b, a))
        maturity.append((T-b))
    else:
        call_prices.append(max(a-k, 0))
        maturity.append(0)

In [ ]:
# Dataframe dos preços
opt_price = pd.DataFrame(call_prices, sims.index)
opt_price = opt_price.rename(columns = {0: "Sim_1_Call"})
opt_price = pd.concat([opt_price, sims["Sim_1"]], axis = 1)
min_max = MinMaxScaler(feature_range=(min(call_prices), max(call_prices)))
opt_price["Sim_1_scaled"] = min_max.fit_transform(opt_price["Sim_1"].values.reshape(-1, 1))
opt_price.index = pd.date_range(start = '01/01/2018', end = '06/01/2018', periods = 1001)

In [ ]:
sns.set(style = "whitegrid", font_scale=2.5)
plt.figure(figsize=(40, 18))
ax = sns.lineplot(data = opt_price[["Sim_1_scaled", "Sim_1_Call"]], palette = "bright", linewidth = 2.7)
ax.set(xlabel = 'Data', ylabel = 'Preços dos Ativos', title = "Preço da Ação - Valor de Venda da Ação")

## Preparação de Dados Para Treinamento

In [ ]:
# Dataframe final
opt_price["Maturity"] = maturity
opt_price["Strike"] = k
opt_price["Risk_Free"] = r
opt_price["Volatility"] = vol
model_data = opt_price.drop(["Sim_1_scaled"], axis = 1)

In [ ]:
# Visualiza
model_data.head()

In [ ]:
# Dados de treino e teste
train_data = model_data.iloc[:round(len(model_data) * 0.8)]  
test_data = model_data.iloc[len(train_data):]

In [ ]:
X_train = train_data.drop(["Sim_1_Call"], axis = 1).values
y_train = train_data["Sim_1_Call"].values

X_test = test_data.drop(["Sim_1_Call"], axis = 1).values
y_test = test_data["Sim_1_Call"].values

min_max = MinMaxScaler()

X_train = min_max.fit_transform(X_train)
X_test = min_max.transform(X_test)

print("X_train shape:", X_train.shape, "\n",
      "y_train shape:", y_train.shape, "\n",
      "X_test shape:", X_test.shape, "\n",
      "y_test shape:", y_test.shape)

## Treinamento

***Treinando Por 10 Epochs***

In [ ]:
%%time

# Treinando Por 10 Epochs

# Modelo
model = Network(mse, mse_prime)
model.add(Dense(5, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 1))

# Treinamento
model.fit(X_train, y_train, epochs = 10, lr = 0.001)

# Previsões
y_pred_10 = model.predict(X_test)
y_pred_10 = [float(a) for a in y_pred_10]

***Treinando Por 100 Epochs***

In [ ]:
%%time

# Treinando Por 100 Epochs

# Modelo
model = Network(mse, mse_prime)
model.add(Dense(5, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 1))

# Treinamento
model.fit(X_train, y_train, epochs = 100, lr = 0.001)

# Previsões
y_pred_100 = model.predict(X_test)
y_pred_100 = [float(a) for a in y_pred_100]

***Treinando Por 200 Epochs***

In [ ]:
%%time

# Treinando Por 200 Epochs

# Modelo
model = Network(mse, mse_prime)
model.add(Dense(5, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 1))

# Treinamento
model.fit(X_train, y_train, epochs = 200, lr = 0.001)

# Previsões
y_pred_200 = model.predict(X_test)
y_pred_200 = [float(a) for a in y_pred_200]

***Treinando Por 1000 Epochs***

In [ ]:
%%time

# Treinando Por 1000 Epochs

# Modelo
model = Network(mse, mse_prime)
model.add(Dense(5, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 1))

# Treinamento
model.fit(X_train, y_train, epochs = 1000, lr = 0.001)

# Previsões
y_pred_1k = model.predict(X_test)
y_pred_1k = [float(a) for a in y_pred_1k]

***Treinando Por 5000 Epochs***

In [ ]:
%%time

# Treinando Por 5000 Epochs

# Modelo
model = Network(mse, mse_prime)
model.add(Dense(5, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 200))
model.add(ActLayer(relu, relu_prime))
model.add(Dense(200, 1))

# Treinamento
model.fit(X_train, y_train, epochs = 5000, lr = 0.001)

# Previsões
y_pred_5k = model.predict(X_test)
y_pred_5k = [float(a) for a in y_pred_5k]

## Testando e Comparando os Resultados

In [ ]:
# Ajusta o shape das perevisões para cada treinamento
y_pred_10 = np.array(y_pred_10).reshape(-1,)
y_pred_100 = np.array(y_pred_100).reshape(-1,)
y_pred_200 = np.array(y_pred_200).reshape(-1,)
y_pred_1k = np.array(y_pred_1k).reshape(-1,)
# y_pred_5k = np.array(y_pred_5k).reshape(-1,)

In [ ]:
# Dataframe das previsões
all_preds = pd.DataFrame({"Valor_Real": y_test,
                          "10 Epochs": y_pred_10,
                          "100 Epochs": y_pred_100,
                          "200 Epochs": y_pred_200,
                          "1000 Epochs": y_pred_1k,
                        #   "5000 Epochs": y_pred_5k
                          }, 
                          index = test_data.index)

In [ ]:
# Plot
plt.figure(figsize = (40, 18))
ax = sns.lineplot(data = all_preds[["Valor_Real", "10 Epochs"]], palette = "bright", linewidth = 2.5)
ax.set(xlabel = 'Data', 
       ylabel = 'Preço do Ativo', 
       title = f'Após 10 Epochs,  MSE:{round(mean_squared_error(all_preds.Valor_Real, all_preds["10 Epochs"]), 3)}')

plt.show()

In [ ]:
# Plot
plt.figure(figsize = (40, 18))
ax = sns.lineplot(data = all_preds[["Valor_Real", "100 Epochs"]], palette = "bright", linewidth = 2.5)
ax.set(xlabel = 'Data', 
       ylabel = 'Preço do Ativo', 
       title = f'Após 100 Epochs,  MSE:{round(mean_squared_error(all_preds.Valor_Real, all_preds["100 Epochs"]), 3)}')
plt.show()

In [ ]:
# Plot
plt.figure(figsize = (40, 18))
ax = sns.lineplot(data = all_preds[["Valor_Real", "200 Epochs"]], palette = "bright", linewidth = 2.5)
ax.set(xlabel = 'Data', 
       ylabel = 'Preço do Ativo', 
       title = f'Após 200 Epochs,  MSE:{round(mean_squared_error(all_preds.Valor_Real, all_preds["200 Epochs"]), 3)}')

plt.show()

In [ ]:
# Plot
plt.figure(figsize = (40, 18))
ax = sns.lineplot(data = all_preds[["Valor_Real", "1000 Epochs"]], palette = "bright", linewidth = 2.5)
ax.set(xlabel = 'Data', 
       ylabel = 'Preço do Ativo', 
       title = f'Após 1000 Epochs,  MSE:{round(mean_squared_error(all_preds.Valor_Real, all_preds["1000 Epochs"]), 3)}')

plt.show()

In [ ]:
# Plot
plt.figure(figsize = (40, 18))
ax = sns.lineplot(data = all_preds[["Valor_Real", "5000 Epochs"]], palette = "bright", linewidth = 2.5)
ax.set(xlabel = 'Data', 
       ylabel = 'Preço do Ativo', 
       title = f'Após 5000 Epochs,  MSE:{round(mean_squared_error(all_preds.Valor_Real, all_preds["5000 Epochs"]), 3)}')

plt.show()

## Conclusão

Treinar o modelo por poucas epochs ou por epochs demais afeta negativamente a performance do modelo. A construção de um modelo equilibrado depende do ponto ideal de treinamento, o que requer experimentação.

# Fim